In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
pip install kaggle


  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)
  Using cached bleach-6.3.0-py3-none-any.whl.metadata (31 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached kaggle-1.7.4.5-py3-none-any.whl (181 kB)
Using cached bleach-6.3.0-py3-none-any.whl (164 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)

   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   -------------------- ------

In [4]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

Dataset URL: https://www.kaggle.com/datasets/anandkumarsahu09/cattle-breeds-dataset
License(s): CC0-1.0
cattle-breeds-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!tar -xf cattle-breeds-dataset.zip


In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [7]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True


In [9]:
for layer in base_model.layers[:100]:
    layer.trainable = False


In [10]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 39s 988ms/step - accuracy: 0.4355 - loss: 1.3846 - val_accuracy: 0.5732 - val_loss: 1.1293 - learning_rate: 1.0000e-04
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 571ms/step - accuracy: 0.7699 - loss: 0.7360 - val_accuracy: 0.6527 - val_loss: 1.0062 - learning_rate: 1.0000e-04
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 573ms/step - accuracy: 0.8442 - loss: 0.5042 - val_accuracy: 0.6695 - val_loss: 0.9532 - learning_rate: 1.0000e-04
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 544ms/step - accuracy: 0.9009 - loss: 0.3418 - val_accuracy: 0.6276 - val_loss: 1.0573 - learning_rate: 1.0000e-04
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 602ms/step - accuracy: 0.9205 - loss: 0.2361 - val_accuracy: 0.6820 - val_loss: 0.9600 - learning_rate: 1.0000e-04
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 554ms/step - accuracy: 0.9536 - loss: 0.1722 - val_accuracy: 0.6904 - val_loss: 0.9614 - learning_rate: 1.0000e-04
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 522ms/step - acc

In [13]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {val_acc*100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 321ms/step - accuracy: 0.7113 - loss: 0.9553
Validation Accuracy after fine-tuning: 71.13%


In [14]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [15]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 470ms/step


array([[6.2073889e-04, 1.3516360e-05, 7.7784105e-05, 9.4111275e-04,
        9.9834681e-01],
       [1.5529016e-02, 1.8245204e-04, 5.0544855e-04, 2.6732199e-03,
        9.8110986e-01],
       [1.2876241e-03, 3.1404802e-06, 3.8546597e-05, 2.0084417e-04,
        9.9846977e-01],
       ...,
       [7.2018552e-01, 1.0206574e-03, 2.5291783e-01, 1.1357446e-02,
        1.4518555e-02],
       [5.6724334e-03, 6.0320021e-05, 3.4175036e-04, 6.9560870e-03,
        9.8696935e-01],
       [2.2602316e-02, 9.1682436e-05, 3.4502841e-04, 1.2902636e-03,
        9.7567070e-01]], dtype=float32)

In [16]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([4, 4, 4, 1, 2, 2, 2, 3, 0, 2, 3, 3, 3, 0, 0, 4, 2, 2, 4, 4, 3, 3,
       4, 4, 4, 4, 0, 1, 2, 4, 3, 1, 0, 1, 4, 0, 3, 0, 2, 4, 4, 0, 2, 3,
       1, 2, 4, 1, 0, 1, 4, 1, 1, 2, 2, 3, 3, 0, 4, 3, 2, 3, 2, 4, 2, 0,
       3, 2, 3, 2, 3, 2, 4, 4, 1, 4, 3, 1, 4, 0, 4, 4, 4, 3, 0, 0, 3, 0,
       0, 2, 2, 3, 4, 1, 4, 3, 1, 4, 2, 4, 3, 3, 4, 1, 0, 4, 2, 1, 4, 4,
       2, 0, 4, 3, 4, 2, 1, 4, 3, 2, 0, 3, 2, 0, 2, 4, 4, 0, 2, 1, 3, 1,
       3, 4, 3, 2, 3, 3, 2, 2, 3, 0, 1, 3, 2, 0, 3, 0, 0, 1, 0, 0, 1, 0,
       1, 2, 2, 0, 3, 2, 0, 3, 1, 0, 4, 0, 0, 0, 3, 4, 1, 2, 2, 2, 2, 0,
       0, 0, 4, 4, 2, 2, 4, 0, 0, 2, 2, 2, 1, 3, 1, 4, 3, 4, 4, 1, 3, 1,
       4, 2, 0, 3, 4, 0, 1, 2, 4, 2, 2, 4, 2, 3, 3, 3, 4, 3, 1, 3, 0, 4,
       3, 4, 1, 2, 0, 4, 1, 4, 3, 1, 0, 2, 0, 4, 2, 4, 0, 4, 4],
      dtype=int64)

In [17]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[ 9  8 10  9 16]
 [ 7  5 11 12 12]
 [ 9  6 11 13 11]
 [14  9 12  6  9]
 [ 7  5  8  8 12]]


In [18]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [19]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.20      0.17      0.18        52
      Brown Swiss cattle       0.15      0.11      0.12        47
Holstein Friesian cattle       0.21      0.22      0.22        50
           Jersey cattle       0.12      0.12      0.12        50
         Red Dane cattle       0.20      0.30      0.24        40

                accuracy                           0.18       239
               macro avg       0.18      0.18      0.18       239
            weighted avg       0.18      0.18      0.18       239



In [20]:
model.save("cattle_breed_model.keras")
